In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpld3
from scipy.interpolate import interp1d
from nonlinear import hammersteinApproximation as hammerstein
from nonlinear import hammersteinLinearApproximation as linearHammer

## System
Define the system and calculate input and output signals

In [ ]:
nsamples = 2**12
noiselevel = 0.25

def m(u):
    # return (1-np.exp(-np.abs(u)))*np.sign(u)
    return (u**2+0.5*u**3)/1.5
    # return u**2
def lbda(n):
    with np.errstate(divide='ignore',invalid='ignore'):
        return np.select([n<0,n==25,n<1000],[0.,0.7,1.0/1.1**n],0)

inputsignal = 2*np.random.rand(nsamples)-1
pureout = np.zeros(nsamples)

for n in range(nsamples):
    pureout[n] = np.sum( m(inputsignal[0:n+1]) * lbda(np.r_[n:-1:-1]) )
    
noiseout = pureout + noiselevel*np.max(np.abs(pureout))*np.random.randn(nsamples)
scale = np.max(np.abs(noiseout))
noiseout = noiseout/scale


## Non-linear approximation
Define parameters for the approximation of the non-linearity, and perform the calculations.
This part uses the raw input and output algorithms, with a rectangular kernel $K$.

For non-linearities which $m(0)=0$ we can shift the approximation by $\mu(0)$ This will be true for all acoustic non-linearities.

In [ ]:
legendreX, legendreY = hammerstein(inputsignal,noiseout,kernel = 'legendre',kernelParam=None)
finIdx = np.isfinite(legendreY)
appScale = np.max(np.abs(legendreY))
legendreFun = interp1d(legendreX[finIdx],legendreY[finIdx]/appScale)

rectangularX, rectangularY = hammerstein(inputsignal,noiseout,kernel = 'rectangular',kernelParam=None)
finIdx = np.isfinite(rectangularY)
appScale = np.max(np.abs(rectangularY))
rectangularFun = interp1d(rectangularX[finIdx],rectangularY[finIdx]/appScale)

plt.scatter(inputsignal,noiseout,marker='.',facecolor='none',edgecolor='gray',alpha=0.5)
plt.plot(legendreX,legendreFun(legendreX),label='Legendre')
plt.plot(rectangularX,rectangularFun(rectangularX),label='Rectangular')
plt.plot(legendreX,m(legendreX),label='True')
plt.legend()
plt.show()
#mpld3.display()

## Linear system approximation
Approximate the parameter $\kappa$, proportional to the dynamic impulse response of the linear system.
This part uses the proposed algorithm without compensating for the non-linear system.

$\kappa$ can be normalized by dividing by the first element. This will give a better appoximation if the original impulse response has unity gain for the first element.

In [ ]:
kappa = linearHammer(inputsignal, noiseout, nonlinearFun = muFun, len = 50)
lbdaApprox = kappa/kappa[0]

plt.semilogy(np.abs(kappa))
plt.semilogy(np.abs(lbdaApprox))
plt.semilogy(np.abs(lbda(np.arange(kappa.size))))
mpld3.display()
#plt.show()



In [ ]:
%qtconsole